<div style="color:red;background-color:black">
Diamond Light Source

<h1 style="color:red;background-color:antiquewhite"> Python Fundamentals: Operator Overloading</h1>  

©2000-20 Chris Seddon 
</div>

## 1
Execute the following cell to activate styling for this tutorial

In [4]:
from IPython.display import HTML
HTML(f"<style>{open('my.css').read()}</style>")

## 2
Along with many other languages, Python allows operators to be overloaded so they will work with objects of our classes.  Special methods of the form "\__xxx__" must be defined in our class in order to make this work.  

In this tutorial we will be working with a "Time" class to illustate techniques.  Our first special mehod is:<pre>\__str__</pre> that allows our objects to work with print: 

In [5]:
class Time:    
    def __init__(self, hrs, min):
        self.hrs = hrs
        self.min = min

    # used by print
    def __str__(self):
        return "Time is: " + str(self.hrs) + " hrs," + str(self.min) + " mins"


t1 = Time(2, 45)
t2 = Time(5, 30)
print(t1)
print(t2)

Time is: 2 hrs,45 mins
Time is: 5 hrs,30 mins


## 3
Often we want to overload the + and minus operators for our objects.  

By convention, when we say:<pre>t1 + t2</pre>
Python translates this into:<pre>t1.\__add__(t2)</pre>
such that "t1" is referenced as "self" and "t2" as "rhs" in the `__add__` method.

In [7]:
class Time:    
    def __init__(self, hrs, min):
        self.hrs = hrs
        self.min = min

    # rhs must be an int or Time
    def __add__(self, rhs):
        hrs = self.hrs + rhs.hrs
        min = self.min + rhs.min
        if min >= 60:
            hrs = hrs + 1
            min = min - 60
        return Time(hrs, min)
    
    def __sub__(self, rhs):
        hrs = self.hrs - rhs.hrs
        min = self.min - rhs.min
        if min < 0:
            hrs = hrs - 1
            min = min + 60
        return Time(hrs, min)
    
    # used by print
    def __str__(self):
        return "Time is: " + str(self.hrs) + " hrs," + str(self.min) + " mins"

t1 = Time(2, 45)
t2 = Time(5, 30)
print(t1 + t2)
print(t2 - t1)

Time is: 8 hrs,15 mins
Time is: 2 hrs,45 mins


## 3
We can extend these methods to cope when we pass in an integer rather than a Time object.  What needs to be done is to check the type of "rhs".  Thus for<pre>t1 + t2 # t1.\__add__(t2)</pre>
"rhs" is a Time object, but for <pre>t1 + 10 # t1.\__add__(10)</pre>
"rhs" is an integer.

In [10]:
class Time:    
    def __init__(self, hrs, min):
        self.hrs = hrs
        self.min = min

    # rhs must be an int or Time
    def __add__(self, rhs):
        if isinstance(rhs, int): rhs = Time(rhs//60, rhs%60)
        if not isinstance(rhs, Time): raise Exception("Incorrect input type")

        hrs = self.hrs + rhs.hrs
        min = self.min + rhs.min
        if min >= 60:
            hrs = hrs + 1
            min = min - 60
        return Time(hrs, min)
    
    # used by print
    def __str__(self):
        return "Time is: " + str(self.hrs) + " hrs," + str(self.min) + " mins"

t1 = Time(2, 45)
t2 = Time(5, 30)
print(t1 + t2)
print(t1 + 135)

Time is: 8 hrs,15 mins
Time is: 5 hrs,0 mins


## 4
Note that in the above, it is the "rhs" that can be a Time object or an ingteger.  Statements like the following:<pre>1 + t1</pre> 
won't work, because Python translates them into<pre>1.\__add__(t1)</pre>
and the int class doesn't know about Time objects.  

However, when the right hand side of an expression is a Time object, but the left hand side is not, we can use reverse add "\__radd__".  In this case, "self" references the right hand side and the second parameter is the left hand side.  Thus<pre>135 + t1</pre>
gets translated to:<pre>t1.\__radd__(135)</pre>

In [6]:
class Time:    
    def __init__(self, hrs, min):
        self.hrs = hrs
        self.min = min

    # rhs must be an int or Time
    def __add__(self, rhs):
        if isinstance(rhs, int): rhs = Time(rhs//60, rhs%60)
        if not isinstance(rhs, Time): raise Exception("Incorrect input type")

        hrs = self.hrs + rhs.hrs
        min = self.min + rhs.min
        if min >= 60:
            hrs = hrs + 1
            min = min - 60
        return Time(hrs, min)
    
    # sel must be an int or Time
    def __radd__(self, lhs):
        return self + lhs
    
    # used by print
    def __str__(self):
        return "Time is: " + str(self.hrs) + " hrs," + str(self.min) + " mins"

t1 = Time(2, 45)
print(135 + t1)

Time is: 5 hrs,0 mins
Time is: 5 hrs,0 mins


## 5
Notice in the above code, we don't duplicate code in "\__radd__", we simply call "\__add__".  <pre>
self + lhs</pre>
is the same as<pre>\__add__(self, lhs)</pre>

Now sometimes we want to increment our object; we can use "\__iadd__" to overload +=.  Our complete example, now looks like:

In [7]:
class Time:    
    def __init__(self, hrs, min):
        self.hrs = hrs
        self.min = min

    # rhs must be an int or Time
    def __add__(self, rhs):
        if isinstance(rhs, int): rhs = Time(0, rhs)
        if not isinstance(rhs, Time): raise Exception("Incorrect input type")

        hrs = self.hrs + rhs.hrs
        min = self.min + rhs.min
        if min >= 60:
            hrs = hrs + 1
            min = min - 60
        return Time(hrs, min)
    
    # only works if rhs is Time
    def __sub__(self, rhs):
        hrs = self.hrs - rhs.hrs
        min = self.min - rhs.min
        if min < 0:
            hrs = hrs - 1
            min = min + 60
        return Time(hrs, min)
    
    # called if Time is on the rhs
    def __radd__(self, lhs):
        return self + lhs
    
    # called for +=
    def __iadd__(self, other):
        self = self + other
        return self
    
    # used by print
    def __str__(self):
        return "Time is: " + str(self.hrs) + " hrs," + str(self.min) + " mins"

t1 = Time(3, 45)
t2 = Time(1, 20)
t1 += t2
print(t1)
print(t2)

Time is: 5 hrs,5 mins
Time is: 1 hrs,20 mins


## 6
We could go on adding special methods to our class; there are many special methods:
<a href="https://docs.python.org/3/reference/datamodel.html">scroll down to: 3.3.8. Emulating numeric types</a>

## 7


## 8


## 9


## 10


## 11


## 12


## 13


## 14


## 15


## 16


## 17


## 18


## 19


## 20
